### 퀴즈를 풀기 위한 스파크 세션을 생성합니다

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

22/07/24 12:44:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 퀴즈 1. 이용자 데이터를 읽어서 user 라는 변수에 담아서 printSchema 및 display 함수를 써서 스키마와 데이터를 출력하세요
* 이용자 파일 위치 : `data/tbl_user.csv` 
* 파일 포맷 : csv
* 읽기 옵션 : inferSchema=true, header=true

In [2]:
user = spark.read.option("inferSchema", "true").option("header", "true").csv(f"{work_data}/tbl_user.csv")
user.printSchema()
display(user)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)



u_id,u_name,u_gender,u_signup
1,정휘센,남,19700808
2,김싸이언,남,19710201
3,박트롬,여,19951030
4,청소기,남,19770329
5,유코드제로,여,20021029
6,윤디오스,남,20040101
7,임모바일,남,20040807
8,조노트북,여,20161201
9,최컴퓨터,남,20201124


### 퀴즈 2. 이용자 데이터프레임을 아래의 조건에 맞도록 변환 후, tbl_user 이라는 변수에 담아서, 스키마와 데이터를 출력하세요
* 컬럼 이름 변경 : u_signup -> u_register
* 컬럼 변경 : u_name 컬럼의 첫 글자를 u_family_name 으로, 나머지 글자 전부를 u_given_name 으로 u_name 컬럼은 제거
* 최종 출력 컬럼 : u_id, u_gender, u_register, u_family_name, u_given_name

In [3]:
# structured api 방식으로 구현이 가능합니다
from pyspark.sql.functions import *
tbl_user = (
    user
    .withColumnRenamed("u_signup", "u_register")
    .withColumn("u_family_name", substring("u_name", 0, 1))
    .withColumn("u_given_name", col("u_name").substr(2, len("u_name")))
    .drop("u_name")
)
tbl_user.printSchema()
display(tbl_user)

root
 |-- u_id: integer (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_register: integer (nullable = true)
 |-- u_family_name: string (nullable = true)
 |-- u_given_name: string (nullable = true)



u_id,u_gender,u_register,u_family_name,u_given_name
1,남,19700808,정,휘센
2,남,19710201,김,싸이언
3,여,19951030,박,트롬
4,남,19770329,청,소기
5,여,20021029,유,코드제로
6,남,20040101,윤,디오스
7,남,20040807,임,모바일
8,여,20161201,조,노트북
9,남,20201124,최,컴퓨터


In [4]:
from pyspark.sql.functions import *

tbl_user = (
    user
    .withColumnRenamed("u_signup", "u_register")
    .withColumn("u_family_name", expr("substr(u_name, 0, 1)"))
    .withColumn("u_given_name", expr("substr(u_name, 2)"))
    .drop("u_name")
)
tbl_user.printSchema()
display(tbl_user)

root
 |-- u_id: integer (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_register: integer (nullable = true)
 |-- u_family_name: string (nullable = true)
 |-- u_given_name: string (nullable = true)



u_id,u_gender,u_register,u_family_name,u_given_name
1,남,19700808,정,휘센
2,남,19710201,김,싸이언
3,여,19951030,박,트롬
4,남,19770329,청,소기
5,여,20021029,유,코드제로
6,남,20040101,윤,디오스
7,남,20040807,임,모바일
8,여,20161201,조,노트북
9,남,20201124,최,컴퓨터


### 퀴즈 3. 매출 데이터를 읽어서 tbl_purchase 이라는 변수에 담고 tbl_user 과 조인하여 fact_daily 변수에 담아서 데이터를 출력하세요
* 조인 조건 : tbl_user.u_id == tbl_purchase.p_uid
* 조인 방식 : tbl_user 테이블을 기준으로 left outer join

In [5]:
tbl_purchase = spark.read.option("inferSchema", "true").option("header", "true").csv(f"{work_data}/tbl_purchase.csv")
tbl_purchase.printSchema()
display(tbl_purchase)

fact_daily = tbl_user.join(tbl_purchase, tbl_user.u_id == tbl_purchase.p_uid, "left_outer")
fact_daily.printSchema()
display(fact_daily)

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



p_time,p_uid,p_id,p_name,p_amount
1603651550,0,1000,GoldStar TV,100000
1603651550,1,2000,LG DIOS,2000000
1603694755,1,2001,LG Gram,1800000
1603673500,2,2002,LG Cyon,1400000
1603652155,3,2003,LG TV,1000000
1603674500,4,2004,LG Computer,4500000
1603665955,5,2001,LG Gram,3500000
1603666155,5,2003,LG TV,2500000


root
 |-- u_id: integer (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_register: integer (nullable = true)
 |-- u_family_name: string (nullable = true)
 |-- u_given_name: string (nullable = true)
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_gender,u_register,u_family_name,u_given_name,p_time,p_uid,p_id,p_name,p_amount
1,남,19700808,정,휘센,1603694755,1,2001,LG Gram,1800000
1,남,19700808,정,휘센,1603651550,1,2000,LG DIOS,2000000
2,남,19710201,김,싸이언,1603673500,2,2002,LG Cyon,1400000
3,여,19951030,박,트롬,1603652155,3,2003,LG TV,1000000
4,남,19770329,청,소기,1603674500,4,2004,LG Computer,4500000
5,여,20021029,유,코드제로,1603666155,5,2003,LG TV,2500000
5,여,20021029,유,코드제로,1603665955,5,2001,LG Gram,3500000
6,남,20040101,윤,디오스,null,null,null,null,null
7,남,20040807,임,모바일,null,null,null,null,null
8,여,20161201,조,노트북,null,null,null,null,null


### 퀴즈 4. fact_daily 데이터를 na 함수를 써서 매출 정보가 널인 경우는 아래의 조건에 맞게 변경하고 fact_daily_na 변수에 담아서 데이터를 출력하세요
* p_time : 0
* p_uid : 0
* p_name : "미확인"
* p_amount : 0

In [6]:
na_fill = {
    "p_time":0,
    "p_uid":0,
    "p_name":"미확인",
    "p_amount":0
}

fact_daily_na = fact_daily.na.fill(na_fill)
fact_daily_na.printSchema()
display(fact_daily_na)

root
 |-- u_id: integer (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_register: integer (nullable = true)
 |-- u_family_name: string (nullable = true)
 |-- u_given_name: string (nullable = true)
 |-- p_time: integer (nullable = false)
 |-- p_uid: integer (nullable = false)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = false)
 |-- p_amount: integer (nullable = false)



u_id,u_gender,u_register,u_family_name,u_given_name,p_time,p_uid,p_id,p_name,p_amount
1,남,19700808,정,휘센,1603694755,1,2001,LG Gram,1800000
1,남,19700808,정,휘센,1603651550,1,2000,LG DIOS,2000000
2,남,19710201,김,싸이언,1603673500,2,2002,LG Cyon,1400000
3,여,19951030,박,트롬,1603652155,3,2003,LG TV,1000000
4,남,19770329,청,소기,1603674500,4,2004,LG Computer,4500000
5,여,20021029,유,코드제로,1603666155,5,2003,LG TV,2500000
5,여,20021029,유,코드제로,1603665955,5,2001,LG Gram,3500000
6,남,20040101,윤,디오스,0,0,null,미확인,0
7,남,20040807,임,모바일,0,0,null,미확인,0
8,여,20161201,조,노트북,0,0,null,미확인,0


### 퀴즈 5. fact_daily_na 데이터프레임을 이용하여 성별 매출 금액의 합계를 출력하세요
* 결과 스키마 : u_gender, count

In [7]:
count_by_gender = fact_daily_na.groupBy("u_gender").agg(sum("p_amount").alias("sum"))
display(count_by_gender)

u_gender,sum
여,7000000
남,9700000


### 퀴즈 6. fact_daily_na 데이터를 json 포맷으로 저장 후, 다시 읽어서 fact_daily_json 변수에 담아 스키마와 데이터를 출력하세요
* 저장 경로 : `data/fact_daily_na/dt=20220727` 
* 저장 포맷 : json
* 저장 옵션 : 덮어쓰기(overwrite)

In [8]:
json_path = f"{work_data}/fact_daily/dt=20220727"
fact_daily_na.write.mode("overwrite").json(json_path)
fact_daily_json = spark.read.json(json_path)
fact_daily_json.printSchema()
display(fact_daily_json)

root
 |-- p_amount: long (nullable = true)
 |-- p_id: long (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_time: long (nullable = true)
 |-- p_uid: long (nullable = true)
 |-- u_family_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_given_name: string (nullable = true)
 |-- u_id: long (nullable = true)
 |-- u_register: long (nullable = true)



p_amount,p_id,p_name,p_time,p_uid,u_family_name,u_gender,u_given_name,u_id,u_register
1800000,2001,LG Gram,1603694755,1,정,남,휘센,1,19700808
2000000,2000,LG DIOS,1603651550,1,정,남,휘센,1,19700808
1400000,2002,LG Cyon,1603673500,2,김,남,싸이언,2,19710201
1000000,2003,LG TV,1603652155,3,박,여,트롬,3,19951030
4500000,2004,LG Computer,1603674500,4,청,남,소기,4,19770329
2500000,2003,LG TV,1603666155,5,유,여,코드제로,5,20021029
3500000,2001,LG Gram,1603665955,5,유,여,코드제로,5,20021029
0,null,미확인,0,0,윤,남,디오스,6,20040101
0,null,미확인,0,0,임,남,모바일,7,20040807
0,null,미확인,0,0,조,여,노트북,8,20161201


### 퀴즈 7. fact_daily_json 데이터를 상품별 매출 금액의 합계를 구하고 최대 매출 상품이름을 확인하세요
* 결과 스키마 : p_name, amount

In [9]:
structured_fact = (
    fact_daily_json
    .groupBy("p_name").agg(sum("p_amount").alias("sum"))
    .orderBy(desc("sum")).limit(1)
)
display(structured_fact)

p_name,sum
LG Gram,5300000


### 퀴즈 8. '퀴즈 7'을 SparkSQL 방식으로 결과를 출력하세요, 이미 SparkSQL 방식으로 풀었다면 Structured API 방식으로 작성하세요
* 결과 스키마 : p_name, amount

In [10]:
fact_daily_json.createOrReplaceTempView("fact_daily_json")
no_1_product_query = """
select p_name, sum(p_amount) as sum
from fact_daily_json
group by p_name
order by sum desc
limit 1
"""
no_1_product = spark.sql(no_1_product_query)
display(no_1_product)

p_name,sum
LG Gram,5300000
